## 2024年 訪日外客数　地域別人数の可視化

## データについて
利用するデータのファイルは２種類
1. foreign_visitors_2024.csv: 訪日外客数
1. inbound_consumption_2024.csv: 消費単価

### 訪日外客数
* 出典：日本政府観光局「JNTO日本の観光統計データ」国籍/目的別 訪日外客数（2004年～2024年）（PDF）
* URL: https://www.jnto.go.jp/statistics/data/visitors-statistics/
* JNTOから外客数データを取得　時系列推移表　国籍/月別 訪日外客数（2004年～2024年）（PDF）からファイルを取得。2024年の観光客などの人数を抽出・加工しました。


### 消費単価
* 出典：「インバウンド消費動向調査（旧 訪日外国人消費動向調査）」（観光庁）
* 「インバウンド消費動向調査（旧 訪日外国人消費動向調査）『2024年年間集計表』」（観光庁）(https://www.mlit.go.jp/kankocho/tokei_hakusyo/gaikokujinshohidoko.html)から抜粋
* 「居住地（21区分）別　費目別購入率および購入者単価」を加工して作成

### 国・地域コード
* ISO3コードを利用する。
* [wiki iso3-code](https://ja.wikipedia.org/wiki/ISO_3166-1)を参考に国・地域名に対応するコードを追記してあります。

In [2]:
import plotly.express as px
import pandas as pd

In [6]:
## 訪日外客数の表示

In [4]:
df = pd.read_csv("./data/foreign_visitors_2024.csv")
df = df.dropna()   # その他の行を削除
df

,地域,人数,観光客,商用客,その他客,コード
0,韓国,8817765,8452682,185779,179304,KOR
1,中国,6981342,6064115,277983,639244,CHN
2,台湾,6044316,5867101,93017,84198,TWN
3,香港,2683391,2637761,23549,22081,HKG
4,タイ,1148848,1092472,25658,30718,THA
5,シンガポール,691226,662890,23334,5002,SGP
6,マレーシア,506883,477923,15718,13242,MYS
7,インドネシア,517651,398949,15452,103250,IDN
8,フィリピン,818659,688452,61237,68970,PHL
9,ベトナム,621173,193117,36066,391990,VNM


### コロプレスマップ 人数で表示
* 国・地域コードを利用して訪日観光客の居住地を塗り分ける
* データセットのコード列で塗り分け
* 人数列をrange_colorの幅でグラデーション
* projection_type = robinson を使用。他　natural earthも良い

In [5]:
# 2. 最小値と最大値を利用してグラデーションの範囲を決める。
# 下限：最小値x0.9
# 上限：最大値x1.1
column_name = "観光客"
margin = 0.1
min_value = df[column_name].min()*(1-margin)
max_value = df[column_name].max()*(1+margin)

# 3. コロプレスマップ
fig = px.choropleth(df, 
                    locations="コード",
                    color=column_name,
                    hover_name="地域",
                    hover_data={column_name: ":,", "コード": False}, # カンマ区切り表示、コードは非表示
                    range_color=[min_value, max_value],              # 色の範囲を指定
                    color_continuous_scale="Rainbow")

# 4. 地図の設定
fig.update_geos(projection_type="robinson",               # 図法。デフォルト：メルカトル図法
                landcolor="rgb(255, 255, 255)",           # 地図の基本色
                projection_rotation=dict(lat=0, lon=148), # 日本を中心に持ってくる
                showcountries=True,                       # 国境描画
                countrycolor="lightgray",                 # 国境線の色。デフォルトは黒っぽい色？
                countrywidth=0.3,                         # 国境線の太さ。デフォルト：0.5        
                )

# 5. 地図のレイアウト設定
fig.update_layout(dragmode=False,
                  width=800,
                  height=550,
                  title=f"2024年 訪日{column_name} 地域別表示")

# 6. 地図表示
fig.show()
